In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [4]:
train_dataset = pd.read_csv('train.csv')
valid_dataset = pd.read_csv('valid.csv')
test_dataset = pd.read_csv('test.csv')

# Label 01

In [5]:
label_1_train_ori = train_dataset.drop(columns=['label_2','label_3','label_4'])

In [6]:
label_1_valid_ori = valid_dataset.drop(columns=['label_2','label_3','label_4'])

## Handle Missing values

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(label_1_train_ori.isnull().sum())

feature_1      0
feature_2      0
feature_3      0
feature_4      0
feature_5      0
feature_6      0
feature_7      0
feature_8      0
feature_9      0
feature_10     0
feature_11     0
feature_12     0
feature_13     0
feature_14     0
feature_15     0
feature_16     0
feature_17     0
feature_18     0
feature_19     0
feature_20     0
feature_21     0
feature_22     0
feature_23     0
feature_24     0
feature_25     0
feature_26     0
feature_27     0
feature_28     0
feature_29     0
feature_30     0
feature_31     0
feature_32     0
feature_33     0
feature_34     0
feature_35     0
feature_36     0
feature_37     0
feature_38     0
feature_39     0
feature_40     0
feature_41     0
feature_42     0
feature_43     0
feature_44     0
feature_45     0
feature_46     0
feature_47     0
feature_48     0
feature_49     0
feature_50     0
feature_51     0
feature_52     0
feature_53     0
feature_54     0
feature_55     0
feature_56     0
feature_57     0
feature_58     0
feature_59    

There's no missing values in label_1

## Before Feature Engineering or Hyperparameter Tuning

In [8]:
X_train_ini = label_1_train_ori.drop(columns=['label_1'])
y_train_ini = label_1_train_ori['label_1']
X_valid_ori = label_1_valid_ori.drop(columns=['label_1'])
y_valid_ori = label_1_valid_ori['label_1']

clf = SVC(kernel='linear')
clf.fit(X_train_ini,y_train_ini)
predictions = clf.predict(X_valid_ori)

In [ ]:
accuracy = accuracy_score(y_valid_ori, predictions)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 96.93%


## PCA Analysis

In [ ]:
PCA_analysis_df = label_1_train_ori.drop(columns=['label_1'])

In [ ]:
PCA_analysis_df_valid = label_1_valid_ori.drop(columns=['label_1'])

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(PCA_analysis_df)

In [ ]:
scaled_valid_data = scaler.transform(PCA_analysis_df_valid)

In [ ]:
pca = PCA(n_components=0.97,svd_solver='full')
principal_components = pca.fit_transform(scaled_data)

In [ ]:
valid_principal_components = pca.transform(scaled_valid_data)

In [ ]:
principal_df_label_1 = pd.DataFrame(data=principal_components, columns=[f'new_feature_{i}' for i in range(principal_components.shape[1])])
valid_principal_df_label_1 = pd.DataFrame(data = valid_principal_components,columns=[f'new_feature_{i}' for i in range(valid_principal_components.shape[1])])

## Hyperparameter Tuning

In [ ]:
param_dist = {
    'C': [0.1,1],           # Uniform distribution between 0 and 10
    'gamma': [0.001, 1],           # Log-uniform distribution between 0.001 and 1
    'kernel': ['rbf'],
    'degree': [1,2]
}

In [ ]:
random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=2,       # Number of parameter settings sampled
    cv=5,             # Number of cross-validation folds
    verbose=1,
    n_jobs=-1         # Use all CPU cores
)

In [ ]:
X = principal_df_label_1
y = label_1_train_ori['label_1']

In [ ]:
random_search.fit(X, y)

In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

## Predicting using a SVC classifier

In [ ]:
y_pred_label_1 = random_search.best_estimator_.predict(valid_principal_df_label_1)
accuracy = accuracy_score(y_pred_label_1, label_1_valid_ori['label_1'])
print(f"Accuracy on validation data: {accuracy*100:.2f}%")

## Make predictions & Test dataset

In [ ]:
data_to_predict = test_dataset.drop(columns=['ID'])
PCA_analysis_df_test_scaled = scaler.transform(data_to_predict)
test_principal_components = pca.transform(PCA_analysis_df_test_scaled)
test_principal_df = pd.DataFrame(data = test_principal_components,columns=[f'new_feature_{i}' for i in range(test_principal_components.shape[1])])

In [ ]:
final_label_1_predict = random_search.best_estimator_.predict(test_principal_df)

In [ ]:
final_submission = "190601D_submission1.csv"
data = {
 'ID' : test_dataset['ID'],
 'label_1' : final_label_1_predict
}
df = pd.DataFrame(data)
df.to_csv(final_submission,index=False)

## Explainability of Results

In [ ]:
def print_top_weight_features(svm_model, n_classes):
  coefficients = svm_model.coef_
  absolute_coefficients = np.abs(coefficients)
  top_weights = []
  n_classes = len(y.unique())
  for class_A in range(n_classes):
    for class_B in range(class_A + 1, n_classes):
        index = int(class_A * (2 * n_classes - class_A - 1) / 2 + class_B - class_A - 1)
        for feature_index, weight in enumerate(coefficients[index]):
            absolute_weight = np.abs(weight)
            if len(top_weights) < 20:
                heapq.heappush(top_weights, (absolute_weight, feature_index, class_A, class_B, weight))
            else:
                if absolute_weight > top_weights[0][0]:
                    heapq.heappop(top_weights)
                    heapq.heappush(top_weights, (absolute_weight, feature_index, class_A, class_B, weight))
  for i, (absolute_weight, feature_index, class_A, class_B, weight) in enumerate(reversed(top_weights)):
    print(f"Top {i + 1}: The weight assigned to feature {feature_index} between class {class_A} and class {class_B} is {weight}")